In [1]:
import random
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from credentials import *
import cufflinks as cf
py.sign_in('ayusharyal','qp72jcx5ac')

In [2]:
class Pkt:
    def __init__(self,pno,sTime,ackTime,ack):
        self.pno = pno #Packet Number
        self.sTime = sTime # Sent Time
        self.ackTime = ackTime # Ack Time Expected
        self.ack = ack # Ack Flag

In [3]:
def createPkt(pno):
    p = Pkt(pno,0,0,False)
    return p

def printPkts(pkts):
    for p in pkts:
        print p.pno,p.sTime,p.ackTime,p.ack

def printPkt(p):
    print p.pno,p.sTime,p.ackTime,p.ack

In [4]:
windowSizeList=range(4,40)
error=1/1000.0
rtt=20
T=100000

throughputs=[]
for windowSize in windowSizeList:
    currentPkts=[]
    windowStart=0
    windowEnd=0
    t=0
    #Get First Set of Packets 
    for t in range(0,windowSize):
        p = createPkt(t)
        currentPkts.append(p)
    windowEnd=windowSize-1
    # printPkts(currentPkts)
    # print t
    #Check acknowledgement and proceed to move window or resend packets
    while  t< T:
        #print t
        for pkt in currentPkts:
            if t== pkt.ackTime:
                if random.uniform(0,1) > error:
                    pkt.ack=True
                else: #resend packet
                    pkt.sTime=t
                    pkt.ackTime=t+rtt
            if pkt.sTime==0:
                #print pkt.pno
                pkt.sTime=t
                pkt.ackTime=t+rtt+currentPkts.index(pkt)
            # if first packet is acknowlegded
        checkFirstPkt=currentPkts[0].ack
        while(checkFirstPkt):
            currentPkts.remove(currentPkts[0])
            currentPkts.append(createPkt(windowEnd+1))
            windowEnd=windowEnd+1
            checkFirstPkt=currentPkts[0].ack
        t+=1
    # print windowEnd
    throughputs.append(windowEnd)


In [5]:
df = pd.DataFrame({'x':windowSizeList , 'y': throughputs})
data = [
        go.Scatter(
            x=df['x'], # assign x as the dataframe column 'x'
            y=df['y']
        )
]
layout = go.Layout(
    title='Plot for SRP ',
    xaxis=dict(
        title='Window Size',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Throughput',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)